In [7]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from pandas import DataFrame
from datetime import datetime
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv1D, MaxPooling1D ,AveragePooling1D
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
#from tensorflow.keras.optimizers import Adam


config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


# 출처: https://3months.tistory.com/206 [Deep Play]

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8061345179840805265
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9232242443
locality {
  bus_id: 1
}
incarnation: 12284719422330649493
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [ ]:
workdir = "path"

In [9]:
dataset = np.load(workdir + "//190613_tcga_stad_10_drugs_input.npz")

In [10]:
dataset.items()

[('x', array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]))]

In [11]:
x = dataset['x']

In [12]:
x.shape

(4410, 23538)

In [13]:
test_X= x

In [14]:
# In[21]:

num_classes = 1
learning_rate = 0.0002
training_epochs = 5
batch_size = 101
img_rows, img_cols = 154, 154


In [15]:
test_X = test_X.reshape(test_X.shape[0], test_X.shape[1], 1)
#input_shape = (img_rows, img_cols, 1)

In [16]:
print(test_X.shape)

(4410, 23538, 1)


In [17]:
test_X = test_X.astype('float32')
# train_X /= 255
# test_X /= 255
print(test_X.shape[0], 'test samples')

4410 test samples


In [18]:
import json
from keras.models import model_from_json, load_model
workdir = "list_path"

In [19]:
json_file = open(workdir+"//MDG_160K_ResNet_Style.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = model_from_json(loaded_model_json)
model = loaded_model

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


Exception ignored in: <bound method NpzFile.__del__ of <numpy.lib.npyio.NpzFile object at 0x000002459B118BE0>>
Traceback (most recent call last):
  File "C:\Users\a\Anaconda3\lib\site-packages\numpy\lib\npyio.py", line 210, in __del__
    self.close()
  File "C:\Users\a\Anaconda3\lib\site-packages\numpy\lib\npyio.py", line 201, in close
    if self.zip is not None:
AttributeError: 'NpzFile' object has no attribute 'zip'


In [20]:
# Option 2: Save/Load the Entire Model
from keras.models import load_model

# Creates a HDF5 file 'my_model.h5'
model.load_weights(workdir + '//MDG_160K_ResNet_Style.h5')

# Deletes the existing model
# del model  

# Returns a compiled model identical to the previous one
# model = load_model('my_model.h5')
model.compile(loss=keras.losses.mean_squared_error,
                  optimizer=keras.optimizers.Adam(learning_rate),
                  metrics=['mse','mae'])

In [21]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             (None, 23538, 1)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 11769, 16)    64          inputs[0][0]                     
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 2353, 16)     0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 2353, 16)     64          max_pooling1d_1[0][0]            
__________________________________________________________________________________________________
activation

In [22]:
import matplotlib
from matplotlib import pyplot as plt
workdir = "C://Users//a//Desktop//Ronny_TF//Ronny_Deep_vs_CDRscan//190613_tcga_stad_epothilone_b_under_deepic50v2"

In [23]:
predicted_value = model.predict(test_X)

In [24]:
predicted_value

array([[-4.9517126],
       [-4.6604624],
       [-5.436971 ],
       ...,
       [ 0.9047848],
       [ 2.0981054],
       [ 0.9047848]], dtype=float32)

In [40]:
table = pd.read_csv(workdir + '//190613_tcga_stad_learning_data_for_10_drugs.csv', index_col='cellLine')

In [32]:
a = pd.DataFrame(predicted_value)
b = pd.DataFrame(table)

In [42]:
a.to_csv(workdir + '//190613_10_drugs_predict_result.csv')